## Getting the data

In [18]:
import pandas as pd
import os
import numpy as np
from scipy import optimize
import numba as nb



In [19]:
%load_ext autoreload
%autoreload 2
from dataframefunctions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
filename = 'germany.csv'
raw = read_csv_to_dataframe(filename)



In [21]:
print(raw)

                                                Party       Date  # of Seats  \
0   AfD - Alternative for Germany (Alternative für...  22-Sep-13           0   
1   AfD - Alternative for Germany (Alternative für...  24-Sep-17          94   
2   CDU - Christian Democratic Union (Christlich D...  16-Oct-94         244   
3   CDU - Christian Democratic Union (Christlich D...  27-Sep-98         198   
4   CDU - Christian Democratic Union (Christlich D...  22-Sep-02         190   
..                                                ...        ...         ...   
89  SPD - Social Democrats (Sozialdemokratische Pa...  22-Sep-02         251   
90  SPD - Social Democrats (Sozialdemokratische Pa...  18-Sep-05         222   
91  SPD - Social Democrats (Sozialdemokratische Pa...  27-Sep-09         146   
92  SPD - Social Democrats (Sozialdemokratische Pa...  22-Sep-13         193   
93  SPD - Social Democrats (Sozialdemokratische Pa...  24-Sep-17         153   

    % of Seats  # of Votes  % of Votes 

In [22]:
ger_data = transform_and_sort_dataframe(raw)

print(ger_data)

    Year Party  # of Seats
2   1994   CDU         244
9   1994   CSU          50
23  1994   FDP          47
32  1994   G -          49
69  1994   PDS          30
87  1994   SPD         252
3   1998   CDU         198
10  1998   CSU          47
24  1998   FDP          43
33  1998   G -          47
70  1998   PDS          36
88  1998   SPD         298
4   2002   CDU         190
11  2002   CSU          58
25  2002   FDP          47
34  2002   G -          55
71  2002   PDS           2
89  2002   SPD         251
5   2005   CDU         180
12  2005   CSU          46
26  2005   FDP          61
35  2005   G -          51
72  2005   PDS          54
90  2005   SPD         222
6   2009   CDU         194
13  2009   CSU          45
27  2009   FDP          93
36  2009   G -          68
47  2009   Lin          76
91  2009   SPD         146
7   2013   CDU         255
14  2013   CSU          56
37  2013   G -          63
48  2013   Lin          64
92  2013   SPD         193
1   2017   AfD          94
8

In [23]:
#get basic variables 
parties_in_year, totalseats_in_year, n_in_year = variables_by_year(ger_data)
print("Parties in Year:", parties_in_year)
print("Total Seats in Year:", totalseats_in_year)
print("Number of Parties in Year:", n_in_year)

Parties in Year: {'1994': ['CDU', 'CSU', 'FDP', 'G -', 'PDS', 'SPD'], '1998': ['CDU', 'CSU', 'FDP', 'G -', 'PDS', 'SPD'], '2002': ['CDU', 'CSU', 'FDP', 'G -', 'PDS', 'SPD'], '2005': ['CDU', 'CSU', 'FDP', 'G -', 'PDS', 'SPD'], '2009': ['CDU', 'CSU', 'FDP', 'G -', 'Lin', 'SPD'], '2013': ['CDU', 'CSU', 'G -', 'Lin', 'SPD'], '2017': ['AfD', 'CDU', 'CSU', 'FDP', 'G -', 'Lin', 'SPD']}
Total Seats in Year: {'1994': 672, '1998': 669, '2002': 603, '2005': 614, '2009': 622, '2013': 631, '2017': 709}
Number of Parties in Year: {'1994': 6, '1998': 6, '2002': 6, '2005': 6, '2009': 6, '2013': 5, '2017': 7}


In [24]:
coalition_combinations = coalition_combinatorics_generator(ger_data,parties_in_year)

for key, value in coalition_combinations.items():
    print(f"{key}: {value}")


('1994', 'CDU'): 244
('1994', 'CSU'): 50
('1994', 'FDP'): 47
('1994', 'G -'): 49
('1994', 'PDS'): 30
('1994', 'SPD'): 252
('1994', 'CDU+CSU'): 294
('1994', 'CDU+FDP'): 291
('1994', 'CDU+G -'): 293
('1994', 'CDU+PDS'): 274
('1994', 'CDU+SPD'): 496
('1994', 'CSU+FDP'): 97
('1994', 'CSU+G -'): 99
('1994', 'CSU+PDS'): 80
('1994', 'CSU+SPD'): 302
('1994', 'FDP+G -'): 96
('1994', 'FDP+PDS'): 77
('1994', 'FDP+SPD'): 299
('1994', 'G -+PDS'): 79
('1994', 'G -+SPD'): 301
('1994', 'PDS+SPD'): 282
('1994', 'CDU+CSU+FDP'): 341
('1994', 'CDU+CSU+G -'): 343
('1994', 'CDU+CSU+PDS'): 324
('1994', 'CDU+CSU+SPD'): 546
('1994', 'CDU+FDP+G -'): 340
('1994', 'CDU+FDP+PDS'): 321
('1994', 'CDU+FDP+SPD'): 543
('1994', 'CDU+G -+PDS'): 323
('1994', 'CDU+G -+SPD'): 545
('1994', 'CDU+PDS+SPD'): 526
('1994', 'CSU+FDP+G -'): 146
('1994', 'CSU+FDP+PDS'): 127
('1994', 'CSU+FDP+SPD'): 349
('1994', 'CSU+G -+PDS'): 129
('1994', 'CSU+G -+SPD'): 351
('1994', 'CSU+PDS+SPD'): 332
('1994', 'FDP+G -+PDS'): 126
('1994', 'FDP+G 

In [25]:
len(coalition_combinations)

473

In [26]:
#get winning coalitions and n from previous combinations
winning_combinations = win_coals_and_parties(coalition_combinations, totalseats_in_year)

(winning_combinations)
#len(coalition_combinations) #test whether still same amount of coalitions

{('1994', 'CDU'): 0,
 ('1994', 'CSU'): 0,
 ('1994', 'FDP'): 0,
 ('1994', 'G -'): 0,
 ('1994', 'PDS'): 0,
 ('1994', 'SPD'): 0,
 ('1994', 'CDU+CSU'): 0,
 ('1994', 'CDU+FDP'): 0,
 ('1994', 'CDU+G -'): 0,
 ('1994', 'CDU+PDS'): 0,
 ('1994', 'CDU+SPD'): 1,
 ('1994', 'CSU+FDP'): 0,
 ('1994', 'CSU+G -'): 0,
 ('1994', 'CSU+PDS'): 0,
 ('1994', 'CSU+SPD'): 0,
 ('1994', 'FDP+G -'): 0,
 ('1994', 'FDP+PDS'): 0,
 ('1994', 'FDP+SPD'): 0,
 ('1994', 'G -+PDS'): 0,
 ('1994', 'G -+SPD'): 0,
 ('1994', 'PDS+SPD'): 0,
 ('1994', 'CDU+CSU+FDP'): 1,
 ('1994', 'CDU+CSU+G -'): 1,
 ('1994', 'CDU+CSU+PDS'): 0,
 ('1994', 'CDU+CSU+SPD'): 1,
 ('1994', 'CDU+FDP+G -'): 1,
 ('1994', 'CDU+FDP+PDS'): 0,
 ('1994', 'CDU+FDP+SPD'): 1,
 ('1994', 'CDU+G -+PDS'): 0,
 ('1994', 'CDU+G -+SPD'): 1,
 ('1994', 'CDU+PDS+SPD'): 1,
 ('1994', 'CSU+FDP+G -'): 0,
 ('1994', 'CSU+FDP+PDS'): 0,
 ('1994', 'CSU+FDP+SPD'): 1,
 ('1994', 'CSU+G -+PDS'): 0,
 ('1994', 'CSU+G -+SPD'): 1,
 ('1994', 'CSU+PDS+SPD'): 0,
 ('1994', 'FDP+G -+PDS'): 0,
 ('199

In [27]:
minimal_winning = min_winning_coals(winning_combinations)
(minimal_winning)

{('1994', 'CDU+SPD'): 1,
 ('1994', 'CDU+CSU+FDP'): 1,
 ('1994', 'CDU+CSU+G -'): 1,
 ('1994', 'CDU+FDP+G -'): 1,
 ('1994', 'CSU+FDP+SPD'): 1,
 ('1994', 'CSU+G -+SPD'): 1,
 ('1994', 'FDP+G -+SPD'): 1,
 ('1998', 'CDU+SPD'): 1,
 ('1998', 'CSU+SPD'): 1,
 ('1998', 'FDP+SPD'): 1,
 ('1998', 'G -+SPD'): 1,
 ('1998', 'CDU+CSU+FDP+G -'): 1,
 ('2002', 'CDU+SPD'): 1,
 ('2002', 'CSU+SPD'): 1,
 ('2002', 'G -+SPD'): 1,
 ('2002', 'CDU+CSU+G -'): 1,
 ('2005', 'CDU+SPD'): 1,
 ('2005', 'CSU+FDP+SPD'): 1,
 ('2005', 'CSU+G -+SPD'): 1,
 ('2005', 'CSU+PDS+SPD'): 1,
 ('2005', 'FDP+G -+SPD'): 1,
 ('2005', 'FDP+PDS+SPD'): 1,
 ('2005', 'G -+PDS+SPD'): 1,
 ('2005', 'CDU+CSU+FDP+G -'): 1,
 ('2005', 'CDU+CSU+FDP+PDS'): 1,
 ('2005', 'CDU+CSU+G -+PDS'): 1,
 ('2005', 'CDU+FDP+G -+PDS'): 1,
 ('2009', 'CDU+SPD'): 1,
 ('2009', 'CDU+CSU+FDP'): 1,
 ('2009', 'CDU+CSU+Lin'): 1,
 ('2009', 'CDU+FDP+G -'): 1,
 ('2009', 'CDU+FDP+Lin'): 1,
 ('2009', 'CDU+G -+Lin'): 1,
 ('2009', 'FDP+Lin+SPD'): 1,
 ('2009', 'CSU+FDP+G -+SPD'): 1,
 

In [29]:
max_loosing = max_loosing_coals(winning_combinations, parties_in_year)

In [32]:
len(minimal_winning)==len(max_loosing) # test whether exactly every MWC has one MLC 

True